# Sandbox

In [1]:
import requests as rq
from bs4 import BeautifulSoup
from collections import defaultdict
import re
import fitz

In [1]:
from dataclasses import asdict
from deuxpots.box import Box, BoxKind, ReferenceBox
from deuxpots.valued_box import ValuedBox
import json

valbox = ValuedBox(
    box=Box(
        code="6FL",
        reference=ReferenceBox(code="6FL", description="Deficits globaux.", type='int'),
        kind=BoxKind.PARTNER_0
    ),
    attribution=.3,
    raw_value=800,
)

asdict(valbox)

{'box': {'code': '6FL',
  'reference': {'code': '6FL',
   'description': 'Deficits globaux.',
   'type': 'int'},
  'kind': <BoxKind.PARTNER_0: 'partner_0'>},
 'raw_value': 800,
 'attribution': 0.3}

In [4]:
PDF_PATH = "test/resources/declaattributionn.pdf"

In [28]:
from deuxpots.pdf_tax_parser import _load_family_box_coords, parse_tax_pdf

family_box_coords = _load_family_box_coords("family_box_coords.json")
parse_tax_pdf(open(PDF_PATH, 'rb').read(), family_box_coords)

{'8HV': 2000,
 '8IV': 800,
 '1AJ': 60000,
 '1BJ': 20000,
 '5HQ': 500,
 'AM': 1,
 'AP': 1,
 'AS': 1,
 'DJ': '1',
 'DN': '1',
 'CF': '2',
 'CG': '1',
 'CH': '1',
 'CI': '1',
 'CR': '2'}

In [10]:
# with fitz.open(PDF_PATH) as doc:
#     for page in doc:
#         text = page.get_text()
#         print(text)

In [3]:
doc = fitz.open(PDF_PATH)

In [14]:
next(pages)

page 2 of declaattributionn.pdf

In [ ]:
def parse_tax_pdf(path):
    box_values = {}
    REGEX_BOX = re.compile(r'\s*([0-9][A-Z]{2})\s+([^:]+):\s([0-9]+)')
    with fitz.open(path) as doc:
        for page in doc:
            text = page.get_text()
            for line in text.split('\n'):
                match = REGEX_BOX.search(line)
                if match:
                    box_code, _, box_value = match.groups()
                    box_values[box_code] = int(box_value)
    return box_values

In [ ]:
parse_tax_pdf(PDF_PATH)

In [ ]:
def compute_tax(data):
    resp = rq.post('https://simulateur-ir-ifi.impots.gouv.fr/cgi-bin/calc-2023.cgi', data=data)
    resp.raise_for_status()
    with open('simulator.html', 'w+') as f:
        f.write(resp.text)
    soup = BeautifulSoup(resp.text)
    results = {input['name']: float(input['value'].replace(' ', '').strip())
               for input in soup.select('input[type=hidden]')}
    try:
        return {
            "Impôt total": results["IINETIR"],
            "Déjà payé": results["IINETIR"] - (results['IINET'] - results['IREST']),
            "Reste à payer": results['IINET'] - results['IREST'],
        }
    except KeyError:
        return {
            "error": soup.select_one('p.margin-left-30px').text
        }

In [32]:
res = parse_tax_pdf(open(PDF_PATH, 'rb').read(), family_box_coords)
res['0DA'] = '1950'

In [34]:
res

{'8HV': 2000,
 '8IV': 800,
 '1AJ': 60000,
 '1BJ': 20000,
 '5HQ': 500,
 'AM': 1,
 'AP': 1,
 'AS': 1,
 'DJ': '1',
 'DN': '1',
 'CF': '2',
 'CG': '1',
 'CH': '1',
 'CI': '1',
 'CR': '2',
 '0DA': '1950'}

In [33]:
from deuxpots.tax_calculator import compute_tax

# compute_tax({
#         '0DA': '1950',
#         'pre_situation_famille': 'M',
#         'pre_situation_residence': 'M',
#         '1AJ': 60000,
#         # '1BJ': 10000,
#         '0AW': 1,
#     })

compute_tax(res)

SimulatorResult(total_tax=15252, already_paid=2800, remains_to_pay=12452)

In [ ]:
LETTER_2_1_MAPPING = letter_mapping = {
    'B': 'A',
    'I': 'H'
}

def convert_2_to_1(box):
    if box == '0DB':
        return '0DA'
    chars = list(box)
    chars[1] = LETTER_2_1_MAPPING.get(chars[1], chars[1])
    return ''.join(chars)

In [ ]:
data_common = {
    '0DA': '1992',  # (M) Date de naissance
    '0DB': '1990',  # (T) Date de naissance
    '1AJ': 357,     # (M) Traitements et salaires
    '1BJ': 59603,   # (T) Traitements et salaires
    '1AP': 5299,    # (M) Autres revenus imposables Chômage, préretraite
    '2DC': 1,       # (M) REVENUS DES VALEURS ET CAPITAUX MOBILIERS
    '2TR': 20,      # (M) REVENUS DES VALEURS ET CAPITAUX MOBILIERS
    '2BH': 21,      # (M) REVENUS DES VALEURS ET CAPITAUX MOBILIERS
    '2CK': 3,       # (M) REVENUS DES VALEURS ET CAPITAUX MOBILIERS
    '5CD': 9,       # (M) Locations meublées cas général - Durée de l'exercice nombre de mois si inférieur à 12.
    '5ND': 4520,    # (M) Locations meublées cas général
    '5HQ': 53083,   # (M) Micro-BNC - Revenus imposables
    '5IQ': 1260,    # (T) Micro-BNC - Revenus imposables
    '8UY': 941,     # (M) Micro-entrepreneur (auto-entrepreneur) : versements d'impôt sur le revenu dont le remboursement est demandé
    '8HV': 387,     # (M) Retenue à la source sur les salaires et pensions 
    '8IV': 9950,    # (T) Retenue à la source sur les salaires et pensions 
    '8HW': 441,     # (M) Acomptes d'impôt sur le revenu 
}

data_single_seed = {
    'pre_situation_famille': 'C',
    'pre_situation_residence': 'M',
}

data_common_seed = {
    'pre_situation_famille': 'O',
    'pre_situation_residence': 'M',
}

data_attribution = {
    '0DA': 'M',
    '0DB': 'T',
    '1AJ': "M",
    '1BJ': "T",
    '1AP': "M",
    '2DC': "M",
    '2TR': "M",
    '2BH': "M",
    '2CK': "M",
    '5CD': "M",
    '5ND': "M",
    '5HQ': "M",
    '5IQ': "T",
    '8UY': "M",
    '8HV': "M",
    '8IV': "T",
    '8HW': "M",
}

assert data_attribution.keys() == data_common.keys()
assert len(set(data_attribution.values())) == 2

In [ ]:
boxes_per_person = defaultdict(list)
for box, who in data_attribution.items():
    boxes_per_person[who].append(box)

results = []
for person, boxes in boxes_per_person.items():
    data = {convert_2_to_1(box): data_common[box] for box in boxes}
    data = {**data, **data_single_seed}
    result = compute_tax(data)
    results.append({"who": person, **result})
    print(data)
result = compute_tax({**data_common, **data_common_seed})
results.append({"who": '+'.join(boxes_per_person), **result})

In [ ]:
attributionA = results[0]['Impôt total'] / (results[0]['Impôt total'] + results[1]['Impôt total'])
attributionB = results[1]['Impôt total'] / (results[0]['Impôt total'] + results[1]['Impôt total'])

In [ ]:
results[0]['A payer (commun)'] = attributionA * results[-1]['Impôt total'] - results[0]['Déjà payé']
results[1]['A payer (commun)'] = attributionB * results[-1]['Impôt total'] - results[1]['Déjà payé']

In [ ]:
for res in results:
    print(res)

In [ ]:
for res in results:
    print(res)

In [ ]:
print("Économie", results[-1]['Impôt total'] - results[0]['Impôt total'] - results[1]['Impôt total'])

# API usage example

In [4]:
import requests
import json
from pathlib import Path

In [8]:
files={
    "tax_pdf": Path("test/resources/declaattributionn.pdf").open('rb'),
}
data = {
    'boxes': "{}"
}

res = requests.post('http://localhost:8888/individualize', data=data, files=files)
res.raise_for_status()
res_dict = res.json()

In [9]:
for box in res_dict['boxes']:
    if box['attribution'] is None:
        print(box['code'], box['description'])

AS Vous ou votre conjoint (même s'il est décédé), âgés de plus de 75 ans, êtes titulaire de la carte du combattant ou d'une pension militaire d'invalidité ou de victime de guerre
DJ Nombre d'enfants majeurs célibataires sans enfant
DN Nombre d'enfants mariés/pacsés et d'enfants non mariés chargés de famille
CF Nombre d'enfants à charge non mariés, qui ne sont pas en résidence alternée, de moins de 18 ans au 1er janvier de l'année de perception des revenus, ou nés durant la même année ou handicapés quel que soit leur âge
CG Nombre d'enfants qui ne sont pas en résidence alternée à charge titulaires de la carte d'invalidité.
CH Nombre d'enfants à charge en résidence alternée, non mariés de moins de 18 ans au 1er janvier de l'année de perception des revenus, ou nés durant la même année ou handicapés quel que soit leur âge
CI Nombre d'enfants à charge en résidence alternée titulaires de la carte d'invalidité
CR Nombre de titulaires (autres que les enfants) de la carte invalidité d'au moins 

In [ ]:
params = res_dict.copy()
for box in params['boxes']:
    if box['attribution'] is None:
        box['attribution'] = 0

In [ ]:
files={
    "tax_pdf": Path("declaattributionn.pdf").open('rb'),
}
data = {
    'boxes': json.dumps(params['boxes'])
}

res = requests.post('http://localhost:8888/individualize', data=data, files=files)
res.raise_for_status()
res_dict = res.json()

In [ ]:
res_dict['individualized']

{'partners': [{'already_paid': 2000,
   'proportion': 0.9888587774766636,
   'remains_to_pay': 6747.444745558567,
   'tax_if_single': 9852,
   'total_tax': 8747.444745558567},
  {'already_paid': 800,
   'proportion': 0.011141222523336344,
   'remains_to_pay': -701.4447455585666,
   'tax_if_single': 111,
   'total_tax': 98.5552544414333}],
 'tax_gain': 1117,
 'total_tax_single': 9963,
 'total_tax_together': 8846}